In [2]:
from jupyter_dash import JupyterDash
from dash import html, dcc, Input, Output, State
from dash import dash_table, callback_context
import numpy as np
#from dash.dependencies import Input, Output
#import dash_html_components as html
#import dash_core_components as dcc
import dash_bootstrap_components as dbc
#from dash.dash_table.Format import Group
#import dash_table
import pandas as pd
import plotly.graph_objects as go
import wpv
#import plotly.express as px

#print([{"name": i, "id": i} for i in df.columns])

#JupyterDash.infer_jupyter_proxy_config()

matmap = {'Low Fe Glass':'nkLowFeGlass.csv',
          'FTO':'nkFTO.csv',
          'MAPI':'nkMAPI.csv',
         }
matoptions = [{'label': i, 'value': matmap[i]} for i in matmap]
print(matoptions)

[{'label': 'Low Fe Glass', 'value': 'nkLowFeGlass.csv'}, {'label': 'FTO', 'value': 'nkFTO.csv'}, {'label': 'MAPI', 'value': 'nkMAPI.csv'}]


In [3]:
#app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
app = JupyterDash(__name__,external_stylesheets=[dbc.themes.JOURNAL])
#app = JupyterDash(__name__,external_stylesheets=[dbc.themes.SANDSTONE])

# Create server variable with Flask server object for use with gunicorn
server = app.server

#ART_fig = 
Glass = wpv.Layer(4000,matmap['Low Fe Glass'],'i')
#FTO = wpv.Layer(0.3,'nkFTO','c')
#MAPI = wpv.Layer(0.06,'nkMAPI','c')
#Ag = wpv.Layer(0.01,'nkAg','c')
#TiO2lowE = wpv.Layer(0.02,'nkTiO2','c')
#EVA = wpv.Layer(1500,'nkEVA','i')


#layers = [Glass,FTO,MAPI,EVA,Glass,TiO2lowE,Ag,TiO2lowE]
layers = [Glass]

stack = wpv.Stack(layers)

num_lams = 200
lambdas = np.linspace(0.3,2.5,num=num_lams)
i_ang = 0
    

app.layout = dbc.Container([
    dcc.Store(id="store"),
    dbc.Container([
        html.H4('PVwindow',className='display-4'),
        html.P(
                "A tool for analyzing semi-transparent photovoltaic devices",
                className="",
        ),
        html.Hr(className="my-2"),
    ]),
            
    dbc.Container(
        [
             
             html.H6('Device Stack',className='display-6'),#,class_name="me-md-1"
             html.Span('Layer:',style={'display':'inline-block','font-size':'22px','width':'20%','height':'36px',"verticalAlign": "center",'text-align':'center'}),
                 dbc.Button('+',color="primary",id='button_add',style={'width':'20%','height':'36px'},n_clicks=0),
             dbc.ButtonGroup([
                 dbc.Button('↑',outline=True,color="primary",id='button_up',n_clicks=0),
                 dbc.Button('↓',outline=True,color="primary",id='button_down',n_clicks=0),
             ],
             style={'width':'40%','height':'36px'}
             ),
             html.Br(),
             dbc.Button('Solve',id='button_solve',class_name='d-grid col-6 mx-auto',color='secondary',n_clicks=0),#style={'width':'60%'}
             html.Br(),
             dash_table.DataTable(
                 columns = [{'name':'Material','id':'Material','presentation': 'dropdown'},
                            {'name':'Thickness [μm]','id':'Thickness [μm]','presentation': 'input'},
                            {'name':'PV','id':'PV','presentation': 'dropdown'}],
                 data=[{'Material':'nkLowFeGlass.csv',
                       'Thickness [μm]':4000,
                       'PV':False}],
                 id = 'layer_table',
                 editable=True,
                 row_selectable='single',
                 row_deletable=True,
                 style_as_list_view=True,
                 style_header={'text-align':'left'},
                 style_data={'text-align':'left'},
                 dropdown={
                    'Material': {
                        'options': matoptions
                    },
                    'PV': {
                         'options': [
                            {'label': 'Yes', 'value': True},
                            {'label': 'No', 'value': False}
                        ]
                    }
                },
                #workaround so bootstrap formatting works with dropdowns... see https://github.com/plotly/dash-table/issues/221
                css=[{"selector": ".Select-menu-outer", "rule": "display: block !important"}],
                 
             ),
            
        ],
        style = {'float':'left','width':'50%','border-right-style':'none','border-width':'thin'},
    ),
    
    dbc.Container(
        [
            html.H6('Analysis',className='display-6'),
            dbc.Tabs(
            [
                dbc.Tab(label="A, R, T", tab_id="rat-tab",tab_style={"marginLeft": "auto"}),
                dbc.Tab(label="Metrics", tab_id="tab-2"),
            ],
            id="tabs",
            active_tab="tab-1",
            ),
            html.Div(id="tab-content", className="p-4"),
        ],
        style = {'float':'left','width':'50%'},
    ),
    ],
    #fluid=True,
)

@app.callback(
    Output("tab-content", "children"),
    [Input("tabs", "active_tab"), Input("store", "data")],
)
def render_tab_content(active_tab, data):
    """
    This callback takes the 'active_tab' property as input, as well as the
    stored graphs, and renders the tab content depending on what the value of
    'active_tab' is.
    """
    if active_tab and data is not None:
        if active_tab == "rat-tab":
            return dcc.Graph(figure=data["scatter"])
        '''
        elif active_tab == "histogram":
            return dbc.Row(
                [
                    dbc.Col(dcc.Graph(figure=data["hist_1"]), width=6),
                    dbc.Col(dcc.Graph(figure=data["hist_2"]), width=6),
                ]
            )
        '''
    return "No tab selected"

@app.callback(
    Output('layer_table', 'data'),
    Input('button_add', 'n_clicks'),
    Input('button_up','n_clicks'),
    Input('button_down','n_clicks'),
    Input('button_solve','n_clicks'),
    State('layer_table', 'data'),
    State('layer_table', 'columns'),
    State('layer_table', 'dropdown'),
    State('layer_table', 'selected_rows')
)
def add_row(n_clicks_add, n_clicks_up, n_clicks_down, n_clicks_solve, data, columns, ddown, srows):
    ctx = callback_context
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]
    #print('button id: ' + str(button_id))
    display(ddown)
    #x = {c['id']: '' for c in columns}
    #print('x=' + str(x))
    
    if n_clicks_add > 0:
        if button_id == 'button_add':
            data.append({c['id']: '' for c in columns})
    
    if n_clicks_up > 0:
        if button_id == 'button_up':
            print('moving row up')
    #print(data)
    
    if n_clicks_solve > 0:
        print('move')
        stack.update_from_dash(data)
        [Rs,As,Ts] = stack.get_specular_RAT(lambdas,i_ang)
    
        fig = go.Figure()
        fig.add_trace(
            go.Scatter(
                x = lambdas,
                y = As,
            )
        )
        
    
    stack.self_summary()
    
    return data

"""
put callback here whose input is the solve button (remove above) and whose output is a figure in the id='store' object
"""
"""
next fix the render_tab_content() one above
"""

if __name__ == '__main__':
    app.run_server(debug=True,mode='inline')

{'Material': {'options': [{'label': 'Low Fe Glass',
    'value': 'nkLowFeGlass.csv'},
   {'label': 'FTO', 'value': 'nkFTO.csv'},
   {'label': 'MAPI', 'value': 'nkMAPI.csv'}]},
 'PV': {'options': [{'label': 'Yes', 'value': True},
   {'label': 'No', 'value': False}]}}

I am a stack with the following layers:
  Layer 1
    Material: nkLowFeGlass.csv
    Thickness: 4000



{'Material': {'options': [{'label': 'Low Fe Glass',
    'value': 'nkLowFeGlass.csv'},
   {'label': 'FTO', 'value': 'nkFTO.csv'},
   {'label': 'MAPI', 'value': 'nkMAPI.csv'}]},
 'PV': {'options': [{'label': 'Yes', 'value': True},
   {'label': 'No', 'value': False}]}}

I am a stack with the following layers:
  Layer 1
    Material: nkLowFeGlass.csv
    Thickness: 4000



{'Material': {'options': [{'label': 'Low Fe Glass',
    'value': 'nkLowFeGlass.csv'},
   {'label': 'FTO', 'value': 'nkFTO.csv'},
   {'label': 'MAPI', 'value': 'nkMAPI.csv'}]},
 'PV': {'options': [{'label': 'Yes', 'value': True},
   {'label': 'No', 'value': False}]}}

I am a stack with the following layers:
  Layer 1
    Material: nkLowFeGlass.csv
    Thickness: 4000



{'Material': {'options': [{'label': 'Low Fe Glass',
    'value': 'nkLowFeGlass.csv'},
   {'label': 'FTO', 'value': 'nkFTO.csv'},
   {'label': 'MAPI', 'value': 'nkMAPI.csv'}]},
 'PV': {'options': [{'label': 'Yes', 'value': True},
   {'label': 'No', 'value': False}]}}

I am a stack with the following layers:
  Layer 1
    Material: nkLowFeGlass.csv
    Thickness: 4000

